In [115]:
import numpy as np
import pandas as pd

In [116]:
import matplotlib.pyplot as plt
from matplotlib import style
import matplotlib.ticker as ticker
import seaborn as sns

In [117]:
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.inspection import permutation_importance
import multiprocessing

In [118]:
labels = pd.read_csv('../csv/train_labels.csv')
labels.head()

,building_id,damage_grade
0,802906,3
1,28830,2
2,94947,3
3,590882,2
4,201944,3


In [119]:
values = pd.read_csv('../csv/train_values.csv')
values.head(10).T

,0,1,2,3,4,5,6,7,8,9
building_id,802906,28830,94947,590882,201944,333020,728451,475515,441126,989500
geo_level_1_id,6,8,21,22,11,8,9,20,0,26
geo_level_2_id,487,900,363,418,131,558,475,323,757,886
geo_level_3_id,12198,2812,8973,10694,1488,6089,12066,12236,7219,994
count_floors_pre_eq,2,2,2,2,3,2,2,2,2,1
age,30,10,10,10,30,10,25,0,15,0
area_percentage,6,8,5,6,8,9,3,8,8,13
height_percentage,5,7,5,5,9,5,4,6,6,4
land_surface_condition,t,o,t,t,t,t,n,t,t,t
foundation_type,r,r,r,r,r,r,r,w,r,i


In [120]:
values.isnull().values.any()

False

In [121]:
labels.isnull().values.any()

False

In [122]:
values.dtypes 

building_id                                int64
geo_level_1_id                             int64
geo_level_2_id                             int64
geo_level_3_id                             int64
count_floors_pre_eq                        int64
age                                        int64
area_percentage                            int64
height_percentage                          int64
land_surface_condition                    object
foundation_type                           object
roof_type                                 object
ground_floor_type                         object
other_floor_type                          object
position                                  object
plan_configuration                        object
has_superstructure_adobe_mud               int64
has_superstructure_mud_mortar_stone        int64
has_superstructure_stone_flag              int64
has_superstructure_cement_mortar_stone     int64
has_superstructure_mud_mortar_brick        int64
has_superstructure_c

In [123]:
values["building_id"].count() == values["building_id"].drop_duplicates().count()

True

In [124]:
values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260601 entries, 0 to 260600
Data columns (total 39 columns):
 #   Column                                  Non-Null Count   Dtype 
---  ------                                  --------------   ----- 
 0   building_id                             260601 non-null  int64 
 1   geo_level_1_id                          260601 non-null  int64 
 2   geo_level_2_id                          260601 non-null  int64 
 3   geo_level_3_id                          260601 non-null  int64 
 4   count_floors_pre_eq                     260601 non-null  int64 
 5   age                                     260601 non-null  int64 
 6   area_percentage                         260601 non-null  int64 
 7   height_percentage                       260601 non-null  int64 
 8   land_surface_condition                  260601 non-null  object
 9   foundation_type                         260601 non-null  object
 10  roof_type                               260601 non-null 

In [125]:
to_be_categorized = ["land_surface_condition", "foundation_type", "roof_type",\
                     "position", "ground_floor_type", "other_floor_type",\
                     "plan_configuration", "legal_ownership_status"]
for row in to_be_categorized:
    values[row] = values[row].astype("category")
values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260601 entries, 0 to 260600
Data columns (total 39 columns):
 #   Column                                  Non-Null Count   Dtype   
---  ------                                  --------------   -----   
 0   building_id                             260601 non-null  int64   
 1   geo_level_1_id                          260601 non-null  int64   
 2   geo_level_2_id                          260601 non-null  int64   
 3   geo_level_3_id                          260601 non-null  int64   
 4   count_floors_pre_eq                     260601 non-null  int64   
 5   age                                     260601 non-null  int64   
 6   area_percentage                         260601 non-null  int64   
 7   height_percentage                       260601 non-null  int64   
 8   land_surface_condition                  260601 non-null  category
 9   foundation_type                         260601 non-null  category
 10  roof_type                       

In [126]:
datatypes = dict(values.dtypes)
for row in values.columns:
    if datatypes[row] != "int64" and datatypes[row] != "int32" and \
       datatypes[row] != "int16" and datatypes[row] != "int8":
        continue
    if values[row].nlargest(1).item() > 32767 and values[row].nlargest(1).item() < 2**31:
        values[row] = values[row].astype(np.int32)
    elif values[row].nlargest(1).item() > 127:
        values[row] = values[row].astype(np.int16)
    else:
        values[row] = values[row].astype(np.int8)
values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260601 entries, 0 to 260600
Data columns (total 39 columns):
 #   Column                                  Non-Null Count   Dtype   
---  ------                                  --------------   -----   
 0   building_id                             260601 non-null  int32   
 1   geo_level_1_id                          260601 non-null  int8    
 2   geo_level_2_id                          260601 non-null  int16   
 3   geo_level_3_id                          260601 non-null  int16   
 4   count_floors_pre_eq                     260601 non-null  int8    
 5   age                                     260601 non-null  int16   
 6   area_percentage                         260601 non-null  int8    
 7   height_percentage                       260601 non-null  int8    
 8   land_surface_condition                  260601 non-null  category
 9   foundation_type                         260601 non-null  category
 10  roof_type                       

In [127]:
labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260601 entries, 0 to 260600
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype
---  ------        --------------   -----
 0   building_id   260601 non-null  int64
 1   damage_grade  260601 non-null  int64
dtypes: int64(2)
memory usage: 4.0 MB


In [128]:
labels["building_id"] = labels["building_id"].astype(np.int32)
labels["damage_grade"] = labels["damage_grade"].astype(np.int8)
labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260601 entries, 0 to 260600
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype
---  ------        --------------   -----
 0   building_id   260601 non-null  int32
 1   damage_grade  260601 non-null  int8 
dtypes: int32(1), int8(1)
memory usage: 1.2 MB


# Primer Modelo

In [129]:
important_values = values\
                .merge(labels, on="building_id")
important_values.drop(columns=["building_id"], inplace = True)
important_values["geo_level_1_id"] = important_values["geo_level_1_id"].astype("category")

In [130]:
important_values.head().T

,0,1,2,3,4
geo_level_1_id,6,8,21,22,11
geo_level_2_id,487,900,363,418,131
geo_level_3_id,12198,2812,8973,10694,1488
count_floors_pre_eq,2,2,2,2,3
age,30,10,10,10,30
area_percentage,6,8,5,6,8
height_percentage,5,7,5,5,9
land_surface_condition,t,o,t,t,t
foundation_type,r,r,r,r,r
roof_type,n,n,n,n,n


In [131]:
X_train, X_test, y_train, y_test = train_test_split(important_values.drop(columns = 'damage_grade'),
                                                    important_values['damage_grade'], test_size = 0.2,
                                                    random_state = 123)

In [132]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res) 

features_to_encode = ["geo_level_1_id", "land_surface_condition", "foundation_type",
                     "roof_type", "position", "ground_floor_type", "other_floor_type",
                     "plan_configuration", "legal_ownership_status"]
for feature in features_to_encode:
    X_train = encode_and_bind(X_train, feature)

In [133]:
rf_model = RandomForestClassifier(n_estimators = 200, max_depth = 25, criterion = "gini", verbose=True, n_jobs=3)
#rf_model.fit(X_train, y_train)

In [134]:
#rf_model.score(X_train, y_train)

In [135]:
#rf_model.feature_importances_

In [136]:
#fig, ax = plt.subplots(figsize = (20,20))
#plt.bar(X_train.columns, rf_model.feature_importances_)
#plt.xlabel("Features")
#plt.xticks(rotation = 90)
#plt.ylabel("Importance")
#plt.show()

# Obtenemos un grafico que indica la importancia de cada columna para el modelo recien generado.

In [137]:
levels = ['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id']
important_columns = [col for col in values.columns if col.startswith('has_superstructure')]
important_values = values[important_columns + levels + \
                          ['building_id', 'foundation_type', 'ground_floor_type', 'age', 'area_percentage']]\
                .merge(labels, on="building_id")
important_values.drop(columns=["building_id"], inplace = True)
for l in levels:
    important_values[l] = important_values[l].astype("category")
important_values.head().T

,0,1,2,3,4
has_superstructure_adobe_mud,1,0,0,0,1
has_superstructure_mud_mortar_stone,1,1,1,1,0
has_superstructure_stone_flag,0,0,0,0,0
has_superstructure_cement_mortar_stone,0,0,0,0,0
has_superstructure_mud_mortar_brick,0,0,0,0,0
has_superstructure_cement_mortar_brick,0,0,0,0,0
has_superstructure_timber,0,0,0,1,0
has_superstructure_bamboo,0,0,0,1,0
has_superstructure_rc_non_engineered,0,0,0,0,0
has_superstructure_rc_engineered,0,0,0,0,0


In [138]:
X_train, X_test, y_train, y_test = train_test_split(important_values,
                                                    important_values['damage_grade'], test_size = 0.2, random_state = 123)

In [139]:
X_train

,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,geo_level_1_id,geo_level_2_id,geo_level_3_id,foundation_type,ground_floor_type,age,area_percentage,damage_grade
103291,0,1,0,0,0,0,1,0,0,0,0,0,1274,4190,r,x,25,8,2
233923,0,1,0,0,0,0,0,0,0,0,0,10,1207,12014,r,f,10,9,2
166653,0,1,0,0,0,0,0,0,0,0,0,17,944,8232,r,f,40,7,3
150634,0,1,0,0,0,0,0,0,0,0,0,10,488,12448,r,f,0,7,3
82720,0,1,0,0,0,0,1,0,0,0,0,26,302,5339,r,f,10,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192476,0,1,0,0,0,0,0,0,0,0,0,8,217,10644,r,x,25,4,3
17730,0,1,0,0,0,0,0,0,0,0,0,8,600,4813,r,f,20,13,3
28030,1,1,0,0,0,0,0,0,0,0,0,8,463,4692,r,f,10,9,3
15725,0,1,0,0,0,0,0,0,0,0,0,8,600,157,r,f,50,5,3


In [140]:
other_levels = ['geo_level_2_id']

def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res) 

def encode_and_bind_mean(df, feature):
    df[feature] = np.where(df.groupby(feature)[feature].transform(len) > 5, df[feature], "other")
    df[feature] = df[feature].map(df.groupby(feature)["damage_grade"].mean().to_dict())
    return(df) 
    
features_to_encode = ["geo_level_1_id", "foundation_type", "ground_floor_type"] 
for feature in features_to_encode:
    X_train = encode_and_bind(X_train, feature)
    X_test = encode_and_bind(X_test, feature)
    
for l in other_levels:
    X_train = encode_and_bind_mean(X_train, l)
    X_test = encode_and_bind_mean(X_test, l)

In [141]:
X_train

,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,...,foundation_type_h,foundation_type_i,foundation_type_r,foundation_type_u,foundation_type_w,ground_floor_type_f,ground_floor_type_m,ground_floor_type_v,ground_floor_type_x,ground_floor_type_z
103291,0,1,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
233923,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
166653,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
150634,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
82720,0,1,0,0,0,0,1,0,0,0,...,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192476,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
17730,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
28030,1,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
15725,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0


In [142]:
rf_model = RandomForestClassifier(n_estimators = 200, max_depth = 25, max_features = 45, criterion = "gini", verbose=True, n_jobs=3)
rf_model.fit(X_train, y_train)

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    5.0s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:   22.7s
[Parallel(n_jobs=3)]: Done 200 out of 200 | elapsed:   23.4s finished


RandomForestClassifier(max_depth=25, max_features=45, n_estimators=200,
                       n_jobs=3, verbose=True)

In [143]:
rf_model.score(X_train, y_train)

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.2s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:    1.1s
[Parallel(n_jobs=3)]: Done 200 out of 200 | elapsed:    1.1s finished


1.0

In [144]:
y_preds = rf_model.predict(X_test)
f1_score(y_test, y_preds, average='micro')

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:    0.2s
[Parallel(n_jobs=3)]: Done 200 out of 200 | elapsed:    0.2s finished


1.0

In [145]:
rf_model.feature_importances_

array([0.00000000e+00, 4.80239270e-05, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 2.03946543e-06, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 2.51533062e-07, 7.03949585e-08, 3.34773332e-02,
       0.00000000e+00, 6.21340108e-05, 5.65209221e-07, 9.59910489e-01,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 2.94638808e-06, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 2.23746681e-03, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.37512449e-06,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.27875480e-03, 2.49473855e-03, 0.00000000e+00, 1.42870105e-06,
       0.00000000e+00, 0.00000000e+00, 4.82382454e-04, 0.00000000e+00,
      

In [146]:
test_values = pd.read_csv('../csv/test_values.csv', index_col = "building_id")
test_values

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
300051,17,596,11307,3,20,7,6,t,r,n,...,0,0,0,0,0,0,0,0,0,0
99355,6,141,11987,2,25,13,5,t,r,n,...,1,0,0,0,0,0,0,0,0,0
890251,22,19,10044,2,5,4,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
745817,26,39,633,1,0,19,3,t,r,x,...,0,0,1,0,0,0,0,0,0,0
421793,17,289,7970,3,15,8,7,t,r,q,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310028,4,605,3623,3,70,20,6,t,r,q,...,1,0,0,0,0,0,0,0,0,0
663567,10,1407,11907,3,25,6,7,n,r,n,...,0,0,0,0,0,0,0,0,0,0
1049160,22,1136,7712,1,50,3,3,t,r,n,...,0,0,0,0,0,0,0,0,0,0


In [147]:
important_columns = [col for col in test_values.columns if col.startswith('has_superstructure')]
test_values_subset = test_values[important_columns + \
                          ['geo_level_1_id', 'foundation_type', 'ground_floor_type', 'age', 'area_percentage']]
test_values_subset["geo_level_1_id"] = test_values_subset["geo_level_1_id"].astype("category")
test_values_subset
#################################################################################################
levels = ['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id']
important_columns = [col for col in test_values.columns if col.startswith('has_superstructure')]
important_values = test_values[important_columns + levels + \
                          ['building_id', 'foundation_type', 'ground_floor_type', 'age', 'area_percentage']]\
                .merge(labels, on="building_id")
important_test_values.drop(columns=["building_id"], inplace = True)
for l in levels:
    important_test_values[l] = important_test_values[l].astype("category")
important_test_values.head().T

KeyError: "['building_id'] not in index"

In [31]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res) 

features_to_encode = ["geo_level_1_id", "foundation_type", "ground_floor_type"]
for feature in features_to_encode:
    test_values_subset = encode_and_bind(test_values_subset, feature)
test_values_subset

,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,...,foundation_type_h,foundation_type_i,foundation_type_r,foundation_type_u,foundation_type_w,ground_floor_type_f,ground_floor_type_m,ground_floor_type_v,ground_floor_type_x,ground_floor_type_z
building_id,,,,,,,,,,,,,,,,,,,,,
300051,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
99355,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
890251,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
745817,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
421793,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310028,0,1,0,0,0,0,1,0,0,0,...,0,0,1,0,0,1,0,0,0,0
663567,1,1,1,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
1049160,0,1,0,0,0,0,1,0,0,0,...,0,0,1,0,0,1,0,0,0,0


In [32]:
preds = rf_model.predict(test_values_subset)

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.3s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:    1.7s
[Parallel(n_jobs=3)]: Done 200 out of 200 | elapsed:    1.8s finished


In [33]:
submission_format = pd.read_csv('../csv/submission_format.csv', index_col = "building_id")

In [34]:
my_submission = pd.DataFrame(data=preds,
                             columns=submission_format.columns,
                             index=submission_format.index)

In [35]:
my_submission.head()

,damage_grade
building_id,
300051,3
99355,2
890251,2
745817,2
421793,3


In [36]:
my_submission.to_csv('../csv/predictions/niyo-model-1-submission.csv')

In [37]:
!head ../csv/predictions/jf-model-1-submission.csv

'head' is not recognized as an internal or external command,
operable program or batch file.
